<a href="https://colab.research.google.com/github/nisrineelhauz/2020-CS109B/blob/master/hw4_problem4_save.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [24]:
%tensorflow_version 2.0.0

`%tensorflow_version` only switches the major version: 1.x or 2.x.
You set: `2.0.0`. This will be interpreted as: `2.x`.


TensorFlow is already loaded. Please restart the runtime to change versions.


In [25]:
%tensorflow_version 2.0
import numpy as np

from PIL import Image

from matplotlib import pyplot
import matplotlib.pylab as plt 
from scipy.signal import convolve2d
%matplotlib inline
plt.style.use('dark_background')
import tensorflow as tf

import os

`%tensorflow_version` only switches the major version: 1.x or 2.x.
You set: `2.0`. This will be interpreted as: `2.x`.


TensorFlow is already loaded. Please restart the runtime to change versions.


In [26]:
#mount your own drive
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
#Getting all pip packages
import subprocess
import sys

reqs = subprocess.check_output([sys.executable, '-m', 'pip', 'freeze'])
installed_packages = [r.decode().split('==') for r in reqs.split()]


In [0]:
#installing tensorflow-datasets==2.1.0
if ['tensorflow-datasets', '2.1.0'] not in installed_packages:
  !pip uninstall tensorflow-datasets -y
  !pip install tensorflow-datasets 

In [0]:
#installing tensorflow-addons==0.6.0
if ['tensorflow-addons', '0.6.0'] not in installed_packages:
  !pip uninstall tensorflow-addons -y
  !pip install tensorflow-addons==0.6.0
  #Please Restart Runtime

In [30]:
#Following creates a directory, downloads the file and unzips it. 
if os.path.isdir('gdrive/My Drive/celeb_a/2.0.0'):
    print('Found dataset' ) #os.listdir('gdrive/My Drive/celeb_a/2.0.0')
else:
    os.makedirs('gdrive/My Drive/celeb_a')
    !wget -O gdrive/My\ Drive/celeb_a/2.0.0.zip https://cs109b2020.s3.amazonaws.com/2.0.0.zip
    !unzip gdrive/My\ Drive/celeb_a/2.0.0.zip  -d gdrive/My\ Drive/celeb_a/

--2020-03-17 17:00:07--  https://cs109b2020.s3.amazonaws.com/2.0.0.zip
Resolving cs109b2020.s3.amazonaws.com (cs109b2020.s3.amazonaws.com)... 52.216.129.171
Connecting to cs109b2020.s3.amazonaws.com (cs109b2020.s3.amazonaws.com)|52.216.129.171|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1313412631 (1.2G) [application/zip]
Saving to: ‘gdrive/My Drive/celeb_a/2.0.0.zip’

gdrive/My Drive/cel 100%[===================>]   1.22G  79.6MB/s    in 16s     

2020-03-17 17:00:23 (77.3 MB/s) - ‘gdrive/My Drive/celeb_a/2.0.0.zip’ saved [1313412631/1313412631]

Archive:  gdrive/My Drive/celeb_a/2.0.0.zip
   creating: gdrive/My Drive/celeb_a/2.0.0/
  inflating: gdrive/My Drive/celeb_a/2.0.0/celeb_a-train.tfrecord-00015-of-00016  
  inflating: gdrive/My Drive/celeb_a/2.0.0/celeb_a-train.tfrecord-00014-of-00016  
  inflating: gdrive/My Drive/celeb_a/2.0.0/celeb_a-train.tfrecord-00008-of-00016  
  inflating: gdrive/My Drive/celeb_a/2.0.0/celeb_a-train.tfrecord-00009-of-0001

In [0]:
import certifi
import urllib3  # For handling https certificate verification 
import scipy.ndimage as ndimage
import tensorflow_datasets as tfds
import tensorflow_addons as tfa


# This line will download the CelebA dataset. The download will only happen the first time you ever run this cell.
train_celeb, test_celeb = tfds.load('celeb_a', split=['train', 'test'], shuffle_files=False, data_dir = 'gdrive/My Drive/', download=False)

In [0]:
# You may use the following two functions
def normalize_image(img):
    return tf.cast(img, tf.float32)/255.

def rot_resize(img, deg):
    rotimg = ndimage.rotate(img, deg, reshape=False, order=3)
    rotimg = np.clip(rotimg, 0., 1.)
    rotimg = tf.image.resize_with_crop_or_pad(rotimg,140,120)
    return rotimg

################################################################
# Don't manually invoke these functions; they are for Dataset 
# pipelining that is already done for you.
################################################################
def tf_rot_resize(img, deg):
    """Dataset pipe that rotates an image and resizes it to 140x120"""
    rotimg = tfa.image.rotate(img, deg/180.*np.pi, interpolation="BILINEAR")
    rotimg = tf.image.resize_with_crop_or_pad(rotimg,140,120)
    return rotimg

def tf_random_rotate_helper(image):
    """Dataset pipe that normalizes image to [0.,1.] and rotates by a random
    amount of degrees in [-60.,60.], returning an (input,target) pair consisting
    of the rotated and resized image and the degrees it has been rotated by."""
    image = normalize_image(image)
    deg = tf.random.uniform([],-60.,60.)
    return (tf_rot_resize(image,deg), deg)  # (data, label)

def tf_random_rotate_image(element):
    """Given an element drawn from the CelebA dataset, this returns a rotated
    image and the amount it has been rotated by, in degrees."""
    image = element['image']
    image, label = tf_random_rotate_helper(image)
    image.set_shape((140,120,3))
    return image, label
################################################################

In [0]:
# Pipeline for creating randomly rotated images with their target labels being 
# the amount they were rotated, in degrees.
train_rot_ds = train_celeb.map(tf_random_rotate_image)
test_rot_ds = test_celeb.map(tf_random_rotate_image)

In [46]:
train_rot_ds.take(4)

<DatasetV1Adapter shapes: ((140, 120, 3), ()), types: (tf.float32, tf.float32)>

In [0]:
#print(tfa.__version__, tf.__version__, tfds.__version__)

In [48]:
train_rot_ds

<DatasetV1Adapter shapes: ((140, 120, 3), ()), types: (tf.float32, tf.float32)>

In [49]:
train_rot_ds.batch(32)

<DatasetV1Adapter shapes: ((None, 140, 120, 3), (None,)), types: (tf.float32, tf.float32)>

In [50]:
!ls gdrive/My\ Drive/celeb_a

2.0.0  2.0.0.zip


In [51]:
!ls gdrive/My\ Drive/celeb_a/2.0.0

celeb_a-test.tfrecord-00000-of-00002
celeb_a-test.tfrecord-00001-of-00002
celeb_a-train.tfrecord-00000-of-00016
celeb_a-train.tfrecord-00001-of-00016
celeb_a-train.tfrecord-00002-of-00016
celeb_a-train.tfrecord-00003-of-00016
celeb_a-train.tfrecord-00004-of-00016
celeb_a-train.tfrecord-00005-of-00016
celeb_a-train.tfrecord-00006-of-00016
celeb_a-train.tfrecord-00007-of-00016
celeb_a-train.tfrecord-00008-of-00016
celeb_a-train.tfrecord-00009-of-00016
celeb_a-train.tfrecord-00010-of-00016
celeb_a-train.tfrecord-00011-of-00016
celeb_a-train.tfrecord-00012-of-00016
celeb_a-train.tfrecord-00013-of-00016
celeb_a-train.tfrecord-00014-of-00016
celeb_a-train.tfrecord-00015-of-00016
celeb_a-validation.tfrecord-00000-of-00002
celeb_a-validation.tfrecord-00001-of-00002
dataset_info.json
image.image.json
